In [61]:
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
# Defining the parameters using CPT
from pgmpy.factors.discrete import TabularCPD

In [88]:
alarm_model = BayesianNetwork(
    [
        ("Burglary", "Alarm"),
        ("Earthquake", "Alarm"),
        ("Alarm", "JohnCalls"),
        ("Alarm", "MaryCalls"),
    ]
)

cpd_burglary = TabularCPD(
    variable="Burglary", variable_card=2, values=[[0.999], [0.001]]
)
cpd_earthquake = TabularCPD(
    variable="Earthquake", variable_card=2, values=[[0.998], [0.002]]
)
cpd_alarm = TabularCPD(
    variable="Alarm",
    variable_card=2,
    values=[[0.999, 0.71, 0.06, 0.05], [0.001, 0.29, 0.94, 0.95]],
    evidence=["Burglary", "Earthquake"],
    evidence_card=[2, 2],
)
cpd_johncalls = TabularCPD(
    variable="JohnCalls",
    variable_card=2,
    values=[[0.95, 0.1], [0.05, 0.9]],
    evidence=["Alarm"],
    evidence_card=[2],
)
cpd_marycalls = TabularCPD(
    variable="MaryCalls",
    variable_card=2,
    values=[[0.99, 0.3], [0.01, 0.7]],
    evidence=["Alarm"],
    evidence_card=[2],
)

# Associating the parameters with the model structure
alarm_model.add_cpds(
    cpd_burglary, cpd_earthquake, cpd_alarm, cpd_johncalls, cpd_marycalls
)

In [89]:
alarm_model.check_model()

True

In [90]:
alarm_model.get_independencies()

(Burglary ⟂ Earthquake)
(Burglary ⟂ MaryCalls, JohnCalls | Alarm)
(Burglary ⟂ JohnCalls | Alarm, MaryCalls)
(Burglary ⟂ MaryCalls | Alarm, JohnCalls)
(Burglary ⟂ MaryCalls, JohnCalls | Alarm, Earthquake)
(Burglary ⟂ JohnCalls | Alarm, MaryCalls, Earthquake)
(Burglary ⟂ MaryCalls | Alarm, JohnCalls, Earthquake)
(MaryCalls ⟂ JohnCalls, Earthquake, Burglary | Alarm)
(MaryCalls ⟂ Earthquake, Burglary | Alarm, JohnCalls)
(MaryCalls ⟂ JohnCalls, Burglary | Alarm, Earthquake)
(MaryCalls ⟂ JohnCalls, Earthquake | Alarm, Burglary)
(MaryCalls ⟂ Burglary | Alarm, JohnCalls, Earthquake)
(MaryCalls ⟂ Earthquake | Alarm, JohnCalls, Burglary)
(MaryCalls ⟂ JohnCalls | Alarm, Earthquake, Burglary)
(JohnCalls ⟂ MaryCalls, Earthquake, Burglary | Alarm)
(JohnCalls ⟂ Earthquake, Burglary | Alarm, MaryCalls)
(JohnCalls ⟂ MaryCalls, Burglary | Alarm, Earthquake)
(JohnCalls ⟂ MaryCalls, Earthquake | Alarm, Burglary)
(JohnCalls ⟂ Burglary | Alarm, MaryCalls, Earthquake)
(JohnCalls ⟂ Earthquake | Alarm, MaryCal

In [117]:
infer = VariableElimination(alarm_model)
q = infer.query(variables=["JohnCalls", "MaryCalls"], evidence={"Alarm": 0, "Burglary": 0, "Earthquake": 0})
print(q)

+--------------+--------------+----------------------------+
| JohnCalls    | MaryCalls    |   phi(JohnCalls,MaryCalls) |
+==============+==============+============================+
| JohnCalls(0) | MaryCalls(0) |                     0.9405 |
+--------------+--------------+----------------------------+
| JohnCalls(0) | MaryCalls(1) |                     0.0095 |
+--------------+--------------+----------------------------+
| JohnCalls(1) | MaryCalls(0) |                     0.0495 |
+--------------+--------------+----------------------------+
| JohnCalls(1) | MaryCalls(1) |                     0.0005 |
+--------------+--------------+----------------------------+


In [119]:
infer = VariableElimination(alarm_model)
m = infer.query(variables=["MaryCalls"], evidence={"Alarm": 0})
j = infer.query(variables=["JohnCalls"], evidence={"Alarm": 0})
a = infer.query(variables=["Alarm"], evidence={"Burglary": 0, "Earthquake": 0})
b = infer.query(variables=["Burglary"])
e = infer.query(variables=["Earthquake"])

In [120]:
print(m, j, a)

+--------------+------------------+
| MaryCalls    |   phi(MaryCalls) |
+==============+==================+
| MaryCalls(0) |           0.9900 |
+--------------+------------------+
| MaryCalls(1) |           0.0100 |
+--------------+------------------+ +--------------+------------------+
| JohnCalls    |   phi(JohnCalls) |
+==============+==================+
| JohnCalls(0) |           0.9500 |
+--------------+------------------+
| JohnCalls(1) |           0.0500 |
+--------------+------------------+ +----------+--------------+
| Alarm    |   phi(Alarm) |
+==========+==============+
| Alarm(0) |       0.9990 |
+----------+--------------+
| Alarm(1) |       0.0010 |
+----------+--------------+


In [112]:
print(b, e)

+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.9990 |
+-------------+-----------------+
| Burglary(1) |          0.0010 |
+-------------+-----------------+ +---------------+-------------------+
| Earthquake    |   phi(Earthquake) |
+===============+===================+
| Earthquake(0) |            0.9980 |
+---------------+-------------------+
| Earthquake(1) |            0.0020 |
+---------------+-------------------+
